In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb, get_bhsa, get_dss

BHSA = get_bhsa()
DSS = get_dss()

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


### 1.Create a list with the verses from the biblical Dead Sea Scrolls

#### 1.1 List of wanted motion verbs

In [5]:
motion_verbs = ['BW>[','HLK[','JY>[','JRD[','<BR[',
                '<LH[','CWB[','>TH[','BRX[','GJX[',
                'GLH[','GLL[','DXP[','DLG[','HWH=[',
                'XWC[','XLP[','XSH[','VB<[','VWF[',
                'MHR[','MWC[','NGC[','NHR[','NWX[',
                'NWS[','NXT[','NVP[','NS<[','NPL[',
                'NTK[','SBB[','SWR[','SLQ[','<WZ[',
                '<WP[','PNH[','PF<[','YWP[','Y<D[',
                'QHL[','QPY[','QRB[','RWY[','FVH[',
                'CWX[','CWR=[','CVP[','CQQ[','T>R[',
                'T<H[']

#### 1.2 List of occurrences of the motion verbs

In [6]:
verbs = TFOb.all("scroll", DSS).filter(biblical=1).to_words.filter_in(lex=motion_verbs)
verbs

<word_5018 "J<WPP TLK JLK J<LHW B>J T<L T<L <LT JRD T<LH TGLH JY> >Y> TPNW GLH MBJ> TLKW [...] B>W JY>W JLK JLK JY> JY> JY> B>W J<LW LK TLKJ HLKTJ TLKJ >LK HLK >LK">

In [7]:
verb = verbs[987]
print(verb)
scroll = verb.to_scrolls.scroll[0]
verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
print(verse)
print(verb.book[0], verb.chapter[0], verb.verse[0], scroll)

B>H
W B>H W HKH >T >RY MYRJM >CR L MWT L MWT W >CR L CBJ L CBJ W >CR L XRB L XRB
Jeremiah 43 11 2Q13


### 2. Used functions

In [8]:
def clean(g_cons):
    """Use to harmonise the DSS content (as strings) with the BHSA content."""
    return g_cons.replace("_", " ").replace("׳", "").replace("'", "")


def find_verb_ref(verb):
    """Returns a list with book, chapter, verse number for a given verb (DSS or BHSA)."""
    if verb.source == "BHSA":
        book = verb.book[0]
        chapter = verb.chapter[0]
        verse_num = verb.verse[0]
    else:
        book = verb.book[0]    
        chapter = verb.chapter[0]
        verse_num = verb.verse[0]
    return [book, chapter, verse_num]


def is_lex_identical(verb_dss): # TODO: handle the "" inside the verse, not only at the end
    """
    Checks if the verses (i.e. BHSA versus DSS) are identical on the lexeme level.
    Remove the empty strings from the DSS verses, if present.
    """
    ref_verb = find_verb_ref(verb_dss)
    
    scroll = verb_dss.to_scrolls.scroll[0]
    
    dss_lex = TFOb.section(ref_verb, DSS, scroll=scroll).to_words.lex
    bhsa_lex = TFOb.section(ref_verb, BHSA).to_words.lex
    
    if dss_lex[-1] == "":
        dss_lex.pop()
    
    return [clean(lex) for lex in bhsa_lex] == [clean(lex) for lex in dss_lex]


def find_bhsa_verb(verb_dss):
    """
    Checks if a verb occurring in DSS also occurs in BHSA (same book, chapter, verse, lexeme).
    Else, returns None.
    """
    
    # Get book chapter verse info from a DSS verb
    ref_dss = find_verb_ref(verb_dss)
    
    if not ref_dss[1].isnumeric():
        # Handles the cases when the chapter in DSS is not a simple number (ex: f14)
        # print("Ref DSS not numeric", ref_dss)
        return 

    # Get the corresponding BHSA verse
    verse_bhsa = TFOb.section(ref_dss, BHSA).to_words
    verb_bhsa = verse_bhsa.filter(lex=verb_dss.lex[0])
    
    # If repetition of verb in same verse: TODO
    if len(verb_bhsa) > 1:
        return # TODO
        scroll = verb_dss.to_scrolls.scroll[0]
        verse_dss = TFOb.section(ref_dss, DSS, scroll)
        print("Verse BHSA:", verse_bhsa)
        print("Verse DSS:", verse_dss)
        
    if verb_bhsa:
        return verb_bhsa
    

def find_clause(verb):
    """Find the complement of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        clause = verb.to_clauses.to_clauses
        return clause
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)
    scroll = verb.to_scrolls.scroll[0]

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb):
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll=scroll).to_words
        clause_bhsa = find_clause(verb_bhsa)
        
    
        first_word_id = clause_bhsa.to_words.ids[0]
        last_word_id = clause_bhsa.to_words.ids[-1]

        verse_ids = clause_bhsa.to_verses.to_words.ids
        
        try: #TODO TODO TODO
            first_word_index = verse_ids.index(first_word_id)
        except:
            print("Case when clause has no verse (to_verses bugs)", verb_bhsa.ids[0])
            return ""

        #first_word_index = verse_ids.index(first_word_id)
        last_word_index = verse_ids.index(last_word_id)
        
        return verse_dss[first_word_index:last_word_index + 1]

    
def find_complements(verb):
    """Find the complement of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        complements = verb.to_clauses.to_phrases.filter(function="Cmpl")
        return complements
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)
    scroll = verb.to_scrolls.scroll[0]

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): # TODO
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll=scroll).to_words
        complements_bhsa = find_complements(verb_bhsa)
        
        complements_dss = []
    
        for complement_bhsa in complements_bhsa:
            first_word_id = complement_bhsa.to_words.ids[0]
            last_word_id = complement_bhsa.to_words.ids[-1]
            
            verse_ids = complement_bhsa.to_verses.to_words.ids
            
            try: #TODO TODO TODO
                first_word_index = verse_ids.index(first_word_id)
            except:
                print("Case when phrase has no verse (to_verses bugs)", verb_bhsa.ids[0])
                return ""
            
            #first_word_index = verse_ids.index(first_word_id)
            last_word_index = verse_ids.index(last_word_id)
            
            complements_dss.append(verse_dss[first_word_index:last_word_index + 1])
        
        return complements_dss

    
def find_subject(verb):
    """Find the subject of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        subjects = verb.to_clauses.to_phrases.filter(function="Subj")
        assert len(subjects) <= 1
        return subjects
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)
    scroll = verb.to_scrolls.scroll[0]

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): # TODO
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll=scroll).to_words
        subject_bhsa = find_subject(verb_bhsa)
        
        if not subject_bhsa: 
            return ""

        first_word_id = subject_bhsa.to_words.ids[0]
        last_word_id = subject_bhsa.to_words.ids[-1]

        verse_ids = subject_bhsa.to_verses.to_words.ids

        first_word_index = verse_ids.index(first_word_id)
        last_word_index = verse_ids.index(last_word_id)
            
        return verse_dss[first_word_index:last_word_index + 1]    
    

def find_prepositions(verb):
    """Find the complement of a verb. If no match, returns None"""
    if verb.source.name == "BHSA":
        complements = verb.to_clauses.to_phrases.filter(function="Cmpl")
        prepositions = complements.to_words.filter(sp="prep")
        return prepositions
    
    # if the verb is not BHSA, it's DSS
    verb_bhsa = find_bhsa_verb(verb)
    scroll = verb.to_scrolls.scroll[0]

    # Check if verses are identical  
    if verb_bhsa and is_lex_identical(verb): 
        verse_dss = TFOb.section(find_verb_ref(verb), DSS, scroll=scroll).to_words
        prepositions_bhsa = find_prepositions(verb_bhsa)
        
        prepositions_dss = []
    
        for preposition_bhsa in prepositions_bhsa:
            first_word_id = preposition_bhsa.to_words.ids[0]
            last_word_id = preposition_bhsa.to_words.ids[-1]
            
            verse_ids = preposition_bhsa.to_verses.to_words.ids
            
            try: #TODO TODO TODO
                first_word_index = verse_ids.index(first_word_id)
            except:
                print("Case when phrase has no verse (to_verses bugs)", verb_bhsa.ids[0])
                return ""
            
            #first_word_index = verse_ids.index(first_word_id)
            last_word_index = verse_ids.index(last_word_id)
            
            prepositions_dss.append(verse_dss[first_word_index:last_word_index + 1])
        
        return prepositions_dss

In [9]:
find_prepositions(verbs[1])

[<word_1 "<L">]

In [10]:
### find_prepositions testing
verb = TFOb(1889904, DSS)
verb.vs
#find_prepositions(verb)

['polel']

### 3. Generate the dataset with pandas

In [11]:
# Create a dataset with the occurrences


items = [] # create an empty list to store all the information for each occ.

       
for verb in verbs:
    
    # Add MT as "scroll" for the BHSA
    if verb.source.name == "BHSA":
        scroll = "MT"
        verse = verb.to_verses
        dir_he_dss_verse = ""
        sign_info = ""

    else:
        scroll = verb.to_scrolls.scroll[0]
        verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
        dir_he_dss_verse = int("H" in verse.uvf_etcbc)
        sign_info = ""
        
        
    subject = find_subject(verb)
    complements = find_complements(verb)
    
    # If complements is None ==> there was no match between DSS and BHSA verses ==> find complement manually
    if complements is None:
        complements = [""]
        dir_he = ""

    for complement in complements:        
        if complement == "":
            dir_he = ""
        else: 
            dir_he = int("H" in complement.to_words.uvf_etcbc)
               
        if verb.g_cons[0] is None:
            g_cons = "no_g_cons"
            #print("Absent G_CONS", verb.ids[0])
        else:
            g_cons = clean(verb.g_cons[0])    
            
        if verse.g_cons is None:
            g_cons_verse = f"LEX: {' '.join(verse.to_words.lex)}"
            #print("Absent G_CONS", verb.ids[0])
        else:
            g_cons_verse = clean(" ".join([g_cons for g_cons in verse.g_cons if g_cons]))
            #print(g_cons_verse)
            
            
        
        # Collect information about the following variables:    
        item = {
            "verb_id": verb.ids[0], 
            "lex": verb.lex[0], 
            "scroll": scroll,
            "book": verb.book[0], 
            "chapter": verb.chapter[0], 
            "verse_num": verb.verse[0],
            "gcons_verb": g_cons,
            "gcons_verse": g_cons_verse,
            "gcons_clause": clean(str(find_clause(verb))),
            "subject": clean(str(subject)),
            "complement": clean(str(complement)),
            "cmpl_lex": "",
            "cmpl_translation": "",
            "dir_he": dir_he,
            "dir_he_dss": dir_he_dss_verse,
            "sign_info": sign_info,
            "stem": verb.vs_etcbc[0],
            "tense": verb.vt_etcbc[0],
            "motion_type": "",
            "comments": "",
        }
        
        if complement != "": 
            prepositions = find_prepositions(verb)
            n = 0
            for preposition in prepositions:
                n += 1
                item[f"preposition_{n}"] = str(preposition.lex[0])

        items.append(item)

Case when clause has no verse (to_verses bugs) 11066
Case when phrase has no verse (to_verses bugs) 97126


In [15]:
df = pd.DataFrame(items).fillna("")
#df.head(5)

In [16]:
filtered_df = df[~((df.preposition_1 == "MN") & (df.preposition_2 == "") & (df.preposition_3 == ""))]
filtered_df = filtered_df[~((filtered_df.preposition_1 == "MN") & (filtered_df.preposition_2 == "MN") & (filtered_df.preposition_3 == ""))]
filtered_df = filtered_df[~((filtered_df.preposition_1 == "MN") & (filtered_df.preposition_2 == "MN") & (filtered_df.preposition_3 == "MN"))]
len(filtered_df)

4060

In [17]:
len(df)

4309

### 4. Save to CSV

In [18]:
filtered_df.to_csv("data/bib_dss_all_verbs_without_min.csv", index=False)